# P1 Edgar data client for REST API

## Initialization

In [8]:
%load_ext autoreload
%autoreload 2

import os
import json
import pprint
from typing import Any

import p1_data_client_python.edgar_client as p1_edg

# Enter your token here.
# You can get your token by signing up at `www.particle.one`.
# P1_API_TOKEN = "YOUR_TOKEN_HERE"
# An example token is like:

P1_API_TOKEN = os.environ["P1_EDGAR_API_TOKEN"]
print("P1_API_TOKEN=", P1_API_TOKEN)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
P1_API_TOKEN= 8c9c9458b145202c7a6b6cceaabd82023e957a46d6cf7061ed8e1c94a168f2fd


## Quick start

There are 3 steps:
1. Get information about company identifiers
2. Get information about financial items available
3. Download data

## Mappers

### GvkCikMapper

It handles CIK <-> GVK transformation.

In [2]:
gvk_mapper = p1_edg.GvkCikMapper(token=P1_API_TOKEN)
gvk_mapper.get_gvk_from_cik(cik=940800, as_of_date="2007-01-18")

,cik,effdate,thrudate,gvk
0,0000940800,2007-01-18T00:00:00,2007-03-14T23:59:59,061411


In [3]:
gvk_mapper.get_cik_from_gvk(gvk=61411, as_of_date="2007-01-18")

,cik,effdate,thrudate,gvk
0,0000940800,2007-01-18T00:00:00,2007-03-14T23:59:59,061411


### ItemMapper

It provides mapping between keywords and description of Compustat items.

In [4]:
item_mapper = p1_edg.ItemMapper(token=P1_API_TOKEN)
item_mapper.get_item_from_keywords(keywords=["short-term", "short term"])

,item,description
0,IVSTCH_YEAR,Short-Term Investments - Change
1,CHE_QUARTER,Cash and Short-Term Investments
2,ALTO_QUARTER,Other Long-term Assets
3,IVLT_QUARTER,Total Long-term Investments
4,DLTT_QUARTER,Long-Term Debt - Total
5,DLTR_YEAR,Long-Term Debt - Reduction
6,DLTIS_YEAR,Long-Term Debt - Issuance


In [5]:
item_mapper.get_mapping()

,item,description
0,ACCHG_QUARTER,Accounting Changes / Cumulative Effect
1,ACOMINC_QUARTER,Accumulated Other Comprehensive Income (Loss)
2,ACO_QUARTER,Current Assets - Other - Total
3,ACT_QUARTER,Current Assets - Total
4,ADRR_QUARTER,ADR Ratio
...,...,...
167,XIDO_QUARTER,Extraordinary Items and Discontinued Operations
168,XINT_QUARTER,Interest and Related Expense- Total
169,XI_QUARTER,Extraordinary Items
170,XRD_QUARTER,Research and Development Expense


## Payload data

### Form8

In [9]:
def _print_payload(payload: str, n: int = 300) -> None:
    print(pprint.pformat(payload)[:n])

def to_str(obj: Any) -> str:
    return json.dumps(obj, indent=4, sort_keys=True)

In [5]:
client = p1_edg.EdgarClient(token=P1_API_TOKEN)

payload = client.get_form8_payload(
    cik=18498, start_date="2020-01-04", end_date="2020-12-04", item="ACT_QUARTER",
)
payload

Cik:   0%|          | 0/1 [00:00<?, ?it/s]
18498: : 0it [00:00, ?it/s]
Cik: 100%|██████████| 1/1 [00:01<00:00,  1.23s/it]


,form_uuid,filing_url,form_publication_timestamp,filing_date,creation_timestamp,cik,ticker,item_name,form_table_row_name,item_value,compustat_timestamp,period_of_report,compustat_coifnd_id,gvk
0,514be1a9-88d1-426b-be7e-d7b5a0ff64e6,https://www.sec.gov/Archives/edgar/data/18498/...,2020-03-12 11:45:58+00:00,2020-03-12 00:00:00+00:00,2020-06-12 23:49:03.580000+00:00,18498,GCO,ACT_QUARTER,Total current assets,508.183,2020-03-12 20:00:00+00:00,2020-01-31 00:00:00+00:00,10099821,5109
1,70a8c7b9-d0a7-4c41-902c-89d84d3b9122,https://www.sec.gov/Archives/edgar/data/18498/...,2020-06-09 11:21:05+00:00,2020-06-09 00:00:00+00:00,2020-12-01 17:13:22.396000+00:00,18498,GCO,ACT_QUARTER,Total current assets,735.008,2020-06-09 20:00:00+00:00,2020-04-30 00:00:00+00:00,10164135,5109
2,30cc5db5-e9ce-4d43-830b-d5d5a94b201a,https://www.sec.gov/Archives/edgar/data/18498/...,2020-09-03 11:30:54+00:00,2020-09-03 00:00:00+00:00,2020-09-03 11:32:58.186000+00:00,18498,GCO,ACT_QUARTER,Total current assets,777.658,NaT,2020-07-31 00:00:00+00:00,,5109
3,9516ca1f-3b90-4c54-b279-7a6cce85ded8,https://www.sec.gov/Archives/edgar/data/18498/...,2020-12-04 12:23:27+00:00,2020-12-04 00:00:00+00:00,2020-12-04 12:40:57.755000+00:00,18498,GCO,ACT_QUARTER,Total current assets,583.958,NaT,2020-10-31 00:00:00+00:00,,5109


In [8]:
client.get_form8_payload(cik=[18498, 319201, 5768])

,form_uuid,filing_url,form_publication_timestamp,filing_date,creation_timestamp,cik,ticker,item_name,form_table_row_name,item_value,compustat_timestamp,period_of_report,compustat_coifnd_id,gvk
0,9f600236-5a62-4861-bcf3-59c2e884cd90,https://www.sec.gov/Archives/edgar/data/5768/0...,2010-02-09T21:17:04,2010-02-09,2020-06-12 22:52:22.626,5768,ASEI,ACO_QUARTER,Other current assets,14.109000000000002,None,2009-12-31T00:00:00,7007593,1554
1,9f600236-5a62-4861-bcf3-59c2e884cd90,https://www.sec.gov/Archives/edgar/data/5768/0...,2010-02-09T21:17:04,2010-02-09,2020-06-12 22:52:22.626,5768,ASEI,AO_QUARTER,Other assets,5.257999999999999,None,2009-12-31T00:00:00,7007593,1554
2,9f600236-5a62-4861-bcf3-59c2e884cd90,https://www.sec.gov/Archives/edgar/data/5768/0...,2010-02-09T21:17:04,2010-02-09,2020-06-12 22:52:22.626,5768,ASEI,INVT_QUARTER,Inventories,47.305,None,2009-12-31T00:00:00,7007593,1554
3,9f600236-5a62-4861-bcf3-59c2e884cd90,https://www.sec.gov/Archives/edgar/data/5768/0...,2010-02-09T21:17:04,2010-02-09,2020-06-12 22:52:22.626,5768,ASEI,LT_QUARTER,Total liabilities,64.296,2010-02-10T05:00:00,2009-12-31T00:00:00,7007593,1554
4,9b8180b0-6fd8-4628-841e-e7340fc3a6fc,https://www.sec.gov/Archives/edgar/data/18498/...,2010-03-04T13:05:44,2010-03-04,2020-06-12 22:52:26.607,18498,GCO,ACO_QUARTER,Other current assets,49.733000000000004,2010-03-05T05:00:00,2010-01-31T00:00:00,7035518,5109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,30cc5db5-e9ce-4d43-830b-d5d5a94b201a,https://www.sec.gov/Archives/edgar/data/18498/...,2020-09-03T11:30:54,2020-09-03,2020-09-03 11:32:58.186,18498,GCO,LCT_QUARTER,Total current liabilities,490.84,None,2020-07-31T00:00:00,,5109
912,30cc5db5-e9ce-4d43-830b-d5d5a94b201a,https://www.sec.gov/Archives/edgar/data/18498/...,2020-09-03T11:30:54,2020-09-03,2020-09-03 11:32:58.186,18498,GCO,LO_QUARTER,Other long-term liabilities,38.552,None,2020-07-31T00:00:00,,5109
913,30cc5db5-e9ce-4d43-830b-d5d5a94b201a,https://www.sec.gov/Archives/edgar/data/18498/...,2020-09-03T11:30:54,2020-09-03,2020-09-03 11:32:58.186,18498,GCO,RECT_QUARTER,Accounts receivable,54.793,None,2020-07-31T00:00:00,,5109
914,3d0e68ab-6e3e-4bd7-8a7f-08634faa33a8,https://www.sec.gov/Archives/edgar/data/319201...,2020-10-28T20:39:33,2020-10-28,2020-10-28 20:40:45.962,319201,KLAC,NI_QUARTER,Net income attributable to KLA,420.56699999999995,2020-10-29T04:00:00,2020-09-30T00:00:00,10256672,6304


### Form10

In [9]:
payload = client.get_form10_payload(
    cik=1002910, start_date="2020-05-11", end_date="2020-05-12",
)

In [10]:
print("len(payload)=%s" % len(payload))
print("payload[0].keys()=%s" % payload[0].keys())

len(payload)=1
payload[0].keys()=dict_keys(['meta', 'data'])


In [11]:
print('payload[0]["meta"]=\n%s' % pprint.pformat(payload[0]["meta"]))

payload[0]["meta"]=
{'cik': {'0': 1002910},
 'filing_date': {'0': '2020-05-11'},
 'filing_url': {'0': 'https://www.sec.gov/Archives/edgar/data/1002910/000100291020000115/0001002910-20-000115-index.html'},
 'form_type': {'0': '10-Q'},
 'uuid': {'0': '981cfc73-7380-4093-8cd7-f0d6a845b3e4'}}


In [12]:
json_str = payload[0]["data"]
print(pprint.pformat(payload[0]["data"])[:2000])

{'calculation': {'calculationLinkbase': [['linkRole',
                                          {'definition': '1002000 - Statement '
                                                         '- Consolidated '
                                                         'Statement of Income '
                                                         '(Loss) and '
                                                         'Comprehensive Income '
                                                         '(Loss)',
                                           'role': 'http://www.ameren.com/role/ConsolidatedStatementOfIncomeLossAndComprehensiveIncomeLoss'},
                                          {},
                                          ['concept',
                                           {'label': 'Net Income (Loss) '
                                                     'Attributable to Parent',
                                            'name': 'us-gaap:NetIncomeLoss'},
                   

### Form4


#### Examples of queries

In [6]:
# Initalize the client.
client = p1_edg.EdgarClient(token=P1_API_TOKEN)

In [14]:
# Get the data for one day and one company.
payload = client.get_form4_payload(
    cik=1524358, start_date="2015-10-23", end_date="2020-10-23",
)
_print_payload(payload)

{'derivative_table': [{'conversion_or_exercise_price': 97.525,
                       'deemed_execution_date': None,
                       'direct_or_indirect_ownership': 'D',
                       'equity_swap_involved': 0,
                       'exercise_date': None,
                       'exp


In [15]:
# Get the data for a week and one company.
payload = client.get_form4_payload(
    cik=1002910, start_date="2015-10-20", end_date="2015-10-27",
)
_print_payload(payload)

{'derivative_table': [],
 'footnotes': [],
 'general_info': [],
 'metadata': [],
 'non_derivative_table': [],
 'reporting_owner_info': []}


In [16]:
# Get the data for a week and list of companies.
payload = client.get_form4_payload(
    cik=[910521, 883241, 80424], start_date="2020-12-10", end_date="2020-12-17",
)
_print_payload(payload)

{'derivative_table': [{'conversion_or_exercise_price': None,
                       'direct_or_indirect_ownership': 'I',
                       'equity_swap_involved': 0,
                       'exercise_date': 'NaT',
                       'expiration_date': 'NaT',
                       'footnote_


In [17]:
# Get the data for one day for all companies.
payload = client.get_form4_payload(
    start_date="2020-12-17", end_date="2020-12-17",
)
_print_payload(payload)

{'derivative_table': [{'conversion_or_exercise_price': None,
                       'direct_or_indirect_ownership': 'D',
                       'equity_swap_involved': 0,
                       'exercise_date': '2020-12-14',
                       'expiration_date': '2020-12-14',
                   


#### Examples, how to handle and show payload data

In [18]:
# Print out a length, and a table names inside a payload.
print("len(payload)=%s" % len(payload))
print("payload.keys()=%s" % payload.keys())

# Show a metadata of a payload.
print('payload["metadata"]=\n%s' % pprint.pformat(payload["metadata"][:2]))

# Print prettified "general_info" table of a payload.
_print_payload(payload["general_info"])

len(payload)=6
payload.keys()=dict_keys(['metadata', 'derivative_table', 'footnotes', 'general_info', 'non_derivative_table', 'reporting_owner_info'])
payload["metadata"]=
[{'cik': 1569187,
  'company': 'Armada Hoffler Properties, Inc.',
  'created_at': '2020-12-17T11:09:21.909919+00:00',
  'edgar_path': 'edgar/data/1569187/000156918720000067/0001569187-20-000067.txt',
  'form_type': '4',
  'payload_path': 's3://edgar-store/form4/2020/ffb0d1daa1928ac1a32cedc45633377e1bac9600.txt.gz',
  'period': '2020-12-17T00:00:00',
  'release_date': '2020-12-17',
  'uuid': '28bf899e-e47d-4063-a9a8-b05a0a5dc571'},
 {'cik': 52827,
  'company': 'RAYONIER INC',
  'created_at': '2020-12-17T11:05:10.096206+00:00',
  'edgar_path': 'edgar/data/52827/000005282720000244/0000052827-20-000244.txt',
  'form_type': '4',
  'payload_path': 's3://edgar-store/form4/2020/cbc1c144ea30740c24386992e348696a74d89f8e.txt.gz',
  'period': '2020-12-17T00:00:00',
  'release_date': '2020-12-17',
  'uuid': '45ab2b89-9a2e-4fba-a5

### Form4 and Form13


#### Examples of queries

In [19]:
# Initalize the client.
client = p1_edg.EdgarClient(token=P1_API_TOKEN)

In [20]:
# Form4. Get the data for one day and one company.
payload = client.get_form4_payload(
    cik=1524358, start_date="2015-10-23", end_date="2020-10-23",
)
_print_payload(payload)

{'derivative_table': [{'conversion_or_exercise_price': 97.525,
                       'deemed_execution_date': None,
                       'direct_or_indirect_ownership': 'D',
                       'equity_swap_involved': 0,
                       'exercise_date': None,
                       'exp


In [21]:
# Form13. Get the data for one day and one company.
payload = client.get_form13_payload(
    cik=1259313, start_date="2015-11-16", end_date="2015-11-16",
)
_print_payload(payload)

{'cover_page': [{'additional_information': None,
                 'amendment_no': None,
                 'amendment_type': None,
                 'conf_denied_expired': None,
                 'date_denied_expired': None,
                 'date_reported': None,
                 'filing_manager_cik': 


In [10]:
# Form13. Get the data for one day and one cusip.
payload = client.get_form13_payload(
    cusip="01449J204", start_date="2015-11-16", end_date="2015-11-16",
)
_print_payload(payload)


CUSIP:   0%|          | 0/1 [00:00<?, ?it/s]
01449J204: : 0it [00:00, ?it/s]
CUSIP:   0%|          | 0/2 [00:00<?, ?it/s]
002824100: : 0it [00:00, ?it/s]
CUSIP:  50%|█████     | 1/2 [02:09<02:09, 129.05s/it]
01449J204: : 0it [00:00, ?it/s]
CUSIP: 100%|██████████| 2/2 [02:45<00:00, 82.56s/it] 


{'cover_page': [{'additional_information': 'Column 7 of the information table '
                                           'identifies the Manager(s) that '
                                           'exercise actual discretion over '
                                           'the relevant reported
{'cover_page': [{'additional_information': None,
                 'amendment_no': None,
                 'amendment_type': None,
                 'conf_denied_expired': None,
                 'date_denied_expired': None,
                 'date_reported': None,
                 'filing_manager_cik': 


In [ ]:
# Form13. Get the data for one day and list of cusip's.
payload = client.get_form13_payload(
    cusip=["002824100", "01449J204"], start_date="2016-11-15", end_date="2016-11-15",
)
_print_payload(payload)

In [22]:
# Form4. Get the data for a week and one company.
payload = client.get_form4_payload(
    cik=1002910, start_date="2015-10-20", end_date="2015-10-27",
)
_print_payload(payload)

{'derivative_table': [],
 'footnotes': [],
 'general_info': [],
 'metadata': [],
 'non_derivative_table': [],
 'reporting_owner_info': []}


In [23]:
# Form4. Get the data for a week and list of companies.
payload = client.get_form4_payload(
    cik=[910521, 883241, 80424], start_date="2020-12-10", end_date="2020-12-17",
)
_print_payload(payload)

{'derivative_table': [{'conversion_or_exercise_price': None,
                       'direct_or_indirect_ownership': 'I',
                       'equity_swap_involved': 0,
                       'exercise_date': 'NaT',
                       'expiration_date': 'NaT',
                       'footnote_


In [24]:
# Form4. Get the data for one day for all companies.
payload = client.get_form4_payload(
    start_date="2020-12-17", end_date="2020-12-17",
)
_print_payload(payload)

{'derivative_table': [{'conversion_or_exercise_price': None,
                       'direct_or_indirect_ownership': 'D',
                       'equity_swap_involved': 0,
                       'exercise_date': '2020-12-14',
                       'expiration_date': '2020-12-14',
                   


#### Examples, how to handle and show payload data

In [25]:
# Print out a length, and a table names inside a payload.
print("len(payload)=%s" % len(payload))
print("payload.keys()=%s" % payload.keys())

len(payload)=6
payload.keys()=dict_keys(['metadata', 'derivative_table', 'footnotes', 'general_info', 'non_derivative_table', 'reporting_owner_info'])


In [26]:
# Show a metadata of a payload.
print('payload["metadata"]=\n%s' % pprint.pformat(payload["metadata"][:50]))

payload["metadata"]=
[{'cik': 1569187,
  'company': 'Armada Hoffler Properties, Inc.',
  'created_at': '2020-12-17T11:09:21.909919+00:00',
  'edgar_path': 'edgar/data/1569187/000156918720000067/0001569187-20-000067.txt',
  'form_type': '4',
  'payload_path': 's3://edgar-store/form4/2020/ffb0d1daa1928ac1a32cedc45633377e1bac9600.txt.gz',
  'period': '2020-12-17T00:00:00',
  'release_date': '2020-12-17',
  'uuid': '28bf899e-e47d-4063-a9a8-b05a0a5dc571'},
 {'cik': 52827,
  'company': 'RAYONIER INC',
  'created_at': '2020-12-17T11:05:10.096206+00:00',
  'edgar_path': 'edgar/data/52827/000005282720000244/0000052827-20-000244.txt',
  'form_type': '4',
  'payload_path': 's3://edgar-store/form4/2020/cbc1c144ea30740c24386992e348696a74d89f8e.txt.gz',
  'period': '2020-12-17T00:00:00',
  'release_date': '2020-12-17',
  'uuid': '45ab2b89-9a2e-4fba-a53c-0f51076307be'},
 {'cik': 1569187,
  'company': 'Armada Hoffler Properties, Inc.',
  'created_at': '2020-12-17T11:09:17.945779+00:00',
  'edgar_path'

In [27]:
# Print prettified "general_info" table of a payload.
json_str = payload["general_info"][:50]
print(pprint.pformat(payload["general_info"]))

[{'document_type': '4',
  'footnote_ids': [],
  'issuer_cik': 1121788,
  'issuer_name': 'GARMIN LTD',
  'issuer_trading_symbol': 'GRMN',
  'period_of_report': '2020-12-15',
  'schema_version': 'X0306',
  'signature_date': '2020-12-17',
  'signature_name': '/s/ Joshua H. Maxfield, Attorney-in-Fact',
  'url': 'https://www.sec.gov/Archives/edgar/data/1121788/000112760220031657/0001127602-20-031657-index.html',
  'uuid': '0138921f-cff1-4fb3-aca1-942f10103af5'},
 {'document_type': '4',
  'footnote_ids': [],
  'issuer_cik': 1597672,
  'issuer_name': 'RAYONIER ADVANCED MATERIALS INC.',
  'issuer_trading_symbol': 'RYAM',
  'period_of_report': '2020-12-14',
  'schema_version': 'X0306',
  'signature_date': '2020-12-17',
  'signature_name': 'Brenda K. Davis, Attorney-in-Fact',
  'url': 'https://www.sec.gov/Archives/edgar/data/1597672/000122520820014793/0001225208-20-014793-index.html',
  'uuid': '023e809e-a3bf-47b9-9d49-cdb055d82896'},
 {'document_type': '4',
  'footnote_ids': [],
  'issuer_cik':